# SfM

In [ ]:
import os

%load_ext autoreload
%autoreload 2

src_path = os.getcwd()
print(src_path)

## Common Configs

In [ ]:
datasets_path = '../data/datasets'
datasets_name = 'television_dataset'
use_masked_images = False

image_dir_path = os.path.join(src_path, datasets_path, datasets_name, 'images')
print(image_dir_path)

## Generate Masks

In [ ]:
import generate_masked_images

In [ ]:
image_masks_path = os.path.join(src_path, datasets_path, datasets_name, 'masks')
masked_image_path = os.path.join(src_path, datasets_path, datasets_name, 'masked_images')
print(image_masks_path, masked_image_path)

In [ ]:
generate_masked_images.main(image_dir_path, image_masks_path, masked_image_path)

## Extract Features

In [ ]:
import extract_features

In [ ]:
superpoint_conf = {
    'output': 'feats-spp',
    'model': {
        'path': 'thirdparty/SuperPointPretrainedNetwork/superpoint_v1.pth',
        'name': 'spp_det'
    },
    'preprocessing': {
        'grayscale': True
    },
    'conf': {
        'descriptor_dim': 256,
        'nms_radius': 3,
        'max_keypoints': 4096,
        'keypoints_threshold': 0.6
    }
}

features_out_path = os.path.join(src_path, datasets_path, datasets_name, superpoint_conf['output'] + '.h5')

print(features_out_path)

In [ ]:
extract_features.main(
    image_dir_path if not use_masked_images else masked_image_path,
    features_out_path,
    superpoint_conf)

## Generate Pairs

In [ ]:
import generate_pairs

In [ ]:
generate_pairs_conf = {
    'method': 'from-poses',
    'num_matched': 10,
    'min_rotation': 10
}

pairs_out_path = os.path.join(
    src_path,
    datasets_path,
    datasets_name,
    '{}-pairs.txt'.format(generate_pairs_conf['method']))
    
print(pairs_out_path)

In [ ]:
generate_pairs.main(
    image_dir_path if not use_masked_images else masked_image_path,
    pairs_out_path,
    generate_pairs_conf)

## Matching Features

In [ ]:
import match_features

In [ ]:
match_features_conf = {
    'model': {
        'path': 'thirdparty/SuperGluePretrainedNetwork/models/weights/superglue_indoor.pth'
    },
    'output': 'matches-spg',
    'conf': {
        'descriptor_dim': 256,
        'weights': 'indoor',
        'match_threshold': 0.7
    }
}

matches_out_path = os.path.join(
    src_path,
    datasets_path,
    datasets_name,
    '{}.h5'.format(match_features_conf['output']))
    
print(matches_out_path)

In [ ]:
match_features.main(
    features_out_path,
    pairs_out_path,
    matches_out_path,
    match_features_conf)

## Generate Empty

In [ ]:
import generate_empty

In [ ]:
sfm_empty_path = os.path.join(src_path, datasets_path, datasets_name, 'sfm_empty')
print(sfm_empty_path)

In [ ]:
generate_empty.main(
    image_dir_path if not use_masked_images else masked_image_path,
    sfm_empty_path)

## Triangulation

In [ ]:
import triangulation

In [ ]:
deep_sfm_path = os.path.join(src_path, datasets_path, datasets_name, 'sfm_ws')
sfm_outputs_path = os.path.join(
    src_path,
    datasets_path,
    datasets_name,
    'outputs' + '_' + superpoint_conf['output'] + '_' + match_features_conf['output'])

print(deep_sfm_path, sfm_outputs_path)

In [ ]:
triangulation.main(
    deep_sfm_path,
    sfm_empty_path,
    sfm_outputs_path,
    pairs_out_path,
    features_out_path,
    matches_out_path,
    image_dir=image_dir_path if not use_masked_images else masked_image_path
)